# Dev: Section

In [44]:
import re
import sys
import logging
from pathlib import Path
sys.path.insert(1, str(Path.cwd().parent)) 


from radreportparser._pattern import _pattern_keys

## Fn: Find Start & End Position

### Start Position

In [78]:
def _find_start_position(text: str, 
                        keys: list[str] | None,
                        word_boundary: bool = True,
                        flags: re.RegexFlag = re.IGNORECASE,
                        ) -> tuple[int, int]:
    """Helper function to find start position of the section"""
    if keys is None:
        return 0, 0
    # Warn if start pattern appears more than once
    for key in keys:
        x = re.findall(key, text, flags)
        count = len(x)
        if count >= 2:
            logging.warning("Start pattern `%s` appear %d times in text, only the first one will be matched.", key, count)
            
    start_match = _pattern_keys(keys, word_boundary, flags).search(text)
    if not start_match:
        return -1, -1  # Indicate no match found
    return start_match.start(), start_match.end()

In [79]:
txt1 = """CT History: A patient presented with these symptoms:
- Chest Pain
- Dyspnea
History: Another History

technique: 

Comparison: None
"""

_find_start_position(txt1, ["history", "technique"])
# _pattern_keys(["history", "technique"]).search(txt1)

(3, 10)

### End Position

In [19]:
def _find_end_position_greedy(text: str, 
                            keys: list[str] | None, 
                            start_pos: int,
                            word_boundary: bool = True,
                            flags: re.RegexFlag = re.IGNORECASE,
                            ) -> int:
    """Find the end position of a section using greedy matching.
    
    Searches for any of the end keys and returns the position of the first match found.
    This is faster but less precise when order matters.
    
    Parameters
    ----------
    text : str
        The input text to search through
    keys : list[str] | None
        List of possible end markers
    start_pos : int
        Position in text to start searching from
    word_boundary : bool, optional
        Whether to use word boundaries in pattern matching
    flags : re.RegexFlag, optional
        Regex flags to use in pattern matching
        
    Returns
    -------
    int
        The ending position in the text
    """
    if keys is None:
        return len(text)
    end_match = _pattern_keys(keys, word_boundary, flags).search(text[start_pos:])
    return len(text) if not end_match else start_pos + end_match.start()


def _find_end_position_sequential(text: str, 
                                keys: list[str] | None, 
                                start_pos: int,
                                word_boundary: bool = True,
                                flags: re.RegexFlag = re.IGNORECASE,
                                ) -> int:
    """Find the end position of a section using sequential matching.
    
    Tries each end key in order and returns the position of the first successful match.
    More precise when the order of keys matters.
    
    Parameters
    ----------
    text : str
        The input text to search through
    keys : list[str] | None
        List of possible end markers, tried in order
    start_pos : int
        Position in text to start searching from
    word_boundary : bool, optional
        Whether to use word boundaries in pattern matching
    flags : re.RegexFlag, optional
        Regex flags to use in pattern matching
        
    Returns
    -------
    int
        The ending position in the text
    """
    if keys is None:
        return len(text)
        
    search_text = text[start_pos:]
    
    # Try each key in sequence
    for key in keys:
        # Create pattern for single key
        pattern = _pattern_keys([key], word_boundary, flags)
        match = pattern.search(search_text)
        
        if match:
            # Return position relative to original text
            return start_pos + match.start()
            
    # If no matches found, return end of text
    return len(text)

In [80]:
txt1 = """CT of the chest
History: A patient presented with these symptoms:
- Chest Pain
- Dyspnea
History: Another

technique: CT chest

Comparison: None
"""

_find_end_position_greedy(txt1, ["history", "technique"], start_pos=1)
_find_end_position_sequential(txt1, ["technique", "history"], start_pos=1)

107

## Main: `extract_section()`

### V2

In [65]:
from typing import Literal

def extract_section(text: str,
                   start_keys: list[str] | None,
                   end_keys: list[str] | None,
                   include_start_keys: bool = False,
                   word_boundary: bool = True,
                   flags: re.RegexFlag = re.IGNORECASE,
                   match_strategy: Literal["greedy", "sequential"] = "greedy",
                   ) -> str | Literal[""]:
    """Extract a section of text between specified start and end keys.
    
    [previous docstring content]
    
    Parameters
    ----------
    [previous parameters]
    match_strategy : MatchStrategy, optional
        Strategy for matching end keys:
        - "greedy": Use first matching end key (faster)
        - "sequential": Try end keys in order (more precise)
        Default is GREEDY
    
    Examples
    --------
    >>> text = "FINDINGS: Normal. TECHNIQUE: MRI. IMPRESSION: Clear."
    >>> # Using sequential matching
    >>> extract_section(text, ["FINDINGS:"], 
    ...                ["TECHNIQUE:", "IMPRESSION:"],
    ...                match_strategy="sequential")
    'Normal.'
    """
    # Find start position
    start_idx_start, start_idx_end = _find_start_position(text, start_keys)
    if start_idx_start == -1:  # No start match found
        return ""
    
    # Find end position based on strategy
    match_strategy_options =  frozenset({"greedy", "sequential"})
    if match_strategy not in match_strategy_options:
        raise ValueError(f"Invalid value: {match_strategy}. Must be one of: {', '.join(match_strategy_options)}")
    
    if match_strategy == "greedy":
        end_idx = _find_end_position_greedy(text, end_keys, start_idx_start)
    else:
        end_idx = _find_end_position_sequential(text, end_keys, start_idx_start)
    
    # Extract the section
    section_start = start_idx_start if include_start_keys else start_idx_end
    return text[section_start:end_idx].strip()

In [81]:
extract_section(txt1, ["history", "technique"], ["comparison"])
# extract_section(txt1, None, ["History"])

': A patient presented with these symptoms:\n- Chest Pain\n- Dyspnea\nHistory: Another\n\ntechnique: CT chest'

In [98]:
extract_section(txt1, 
                start_keys=[r"history\W*"], 
                end_keys=[r"Comparison\W*"], 
                word_boundary= False,
                include_start_keys=False)

'A patient presented with these symptoms:\n- Chest Pain\n- Dyspnea\nHistory: Another\n\ntechnique: CT chest'

In [91]:
t_rep_md1 = """
History: A patient presented with these symptoms:
- Chest Pain
- Dyspnea

**technique:** CT chest
"""

extract_section(t_rep_md1, 
                start_keys=[r"\W*History\W*"], 
                end_keys=[r"\W*technique\W*"],
                word_boundary= True,
                include_start_keys=False)

'A patient presented with these symptoms:\n- Chest Pain\n- Dyspnea'

In [ ]:
t_rep_md2 = """
Finding: 
- A
- B
- C

**Impression:**
- D
- E
- F
"""

extract_section(t_rep_md2, 
                start_keys=[r"\W*Finding(s?)\W*"], 
                end_keys=[r"\W*Impression\W*"],
                word_boundary= True,
                include_start_keys=False)

'A\n- B\n- C'

In [ ]:
t_rep_md3 = """
Clinical Indications: 
- A
- B
- C

**Impression:**
- D
- E
- F
"""

extract_section(t_rep_md3, 
                start_keys= [r"\W*history\W*", r"\W*indication(s?)\W*", *[rf"\W*clinical\s+{h}\W*" for h in ["history", r"indication(s?)"]]],
                end_keys=[r"\W*Impression\W*"],
                word_boundary= True,
                include_start_keys=False)

'A\n- B\n- C'

In [ ]:
## Title
t_rep_md4 = """
CT CHEST WITH CONTRAST
CT WHOLE ABODMEN

History: blah blah blah

Comparison: None

Impression: blah blah blah
"""

extract_section(t_rep_md4, 
                start_keys= None,
                end_keys=[r"\W*History\W*"],
                word_boundary= True,
                include_start_keys=False)


'CT CHEST WITH CONTRAST\nCT WHOLE ABODMEN'

In [ ]:
## Title
t_rep_md4 = """
CT CHEST WITH CONTRAST
CT WHOLE ABODMEN

Blah
"""

extract_section(t_rep_md4, 
                start_keys= None,
                end_keys=[r"\W*Impression\W*"],
                word_boundary= True,
                include_start_keys=False)


'CT CHEST WITH CONTRAST\nCT WHOLE ABODMEN\n\nBlah'

## Deprecated

In [38]:
from typing import Literal

def extract_section2(text: str,
                   start_keys: list[str] | None,
                   end_keys: list[str] | None,
                   include_start_keys: bool = False,
                   word_boundary: bool = True,
                   flags: re.RegexFlag = re.IGNORECASE,
                   ) -> str | Literal[""]:

    # Find start position
    start_idx_start, start_idx_end = _find_start_position(text, start_keys)
    if start_idx_start == -1:  # No start match found
        return ""
    
    # Find end position
    end_idx = _find_end_position(text, end_keys, start_idx_start)
    
    # Extract the section
    section_start = start_idx_start if include_start_keys else start_idx_end
    return text[section_start:end_idx].strip()